<a href="https://colab.research.google.com/github/likit/Advanced-community-health-course/blob/main/duckdb_in_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import duckdb

In [ ]:
conn = duckdb.connect(database=':memory:', read_only=False)

In [ ]:
conn.execute("""
CREATE TABLE users (
  id INTEGER,
  name VARCHAR,
  email VARCHAR,
  join_date DATE,
)
""")

In [ ]:
conn.execute("""
INSERT INTO users (id, name, email, join_date) VALUES
(1, 'Alice', 'alice@example.com', '2021-01-01'),
(2, 'Bob', 'bob@example.com', '2021-02-01')
""")

In [ ]:
result = conn.execute('SELECT * FROM users;').fetchall()

In [ ]:
result

[(1, 'Alice', 'alice@example.com', datetime.date(2021, 1, 1)),
 (2, 'Bob', 'bob@example.com', datetime.date(2021, 2, 1))]

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('2011-2012-health-data.csv')

In [ ]:
data.head()

,ServiceNo,ServiceDate,CustID,age,sex,cmsCode,Weight,Height,HeartRate,Systolic,Diastolic,BMI,HDL,TG,UA
0,201101050001,5/1/2011 0:00,225.0,59.0,2.0,38468.0,81.8,159,78,150,96.0,32.35631502,56.4,115.0,5.04
1,201101050002,5/1/2011 0:00,225.0,55.0,2.0,0.0,53.7,156,72,120,70.0,22.06607495,73.7,141.0,3.82
2,201101050003,5/1/2011 0:00,225.0,60.0,2.0,0.0,57.2,156,72,110,70.0,23.5042735,77.5,69.0,5.01
3,201101050004,5/1/2011 0:00,225.0,55.0,2.0,0.0,60,157,76,120,80.0,24.34175829,77.9,79.0,2.83
4,201101050005,5/1/2011 0:00,225.0,59.0,1.0,0.0,79.6,171.5,68,130,70.0,27.06355345,60.4,94.0,5.82


In [ ]:
duckdb.sql('SELECT * FROM data WHERE age > 60;')

┌──────────────┬─────────────────┬────────┬────────┬────────┬───┬───────────┬─────────────┬─────────┬────────┬────────┐
│  ServiceNo   │   ServiceDate   │ CustID │  age   │  sex   │ … │ Diastolic │     BMI     │   HDL   │   TG   │   UA   │
│    int64     │     varchar     │ double │ double │ double │   │  double   │   varchar   │ varchar │ double │ double │
├──────────────┼─────────────────┼────────┼────────┼────────┼───┼───────────┼─────────────┼─────────┼────────┼────────┤
│ 201101050010 │ 5/1/2011 0:00   │  225.0 │   61.0 │    2.0 │ … │      80.0 │ 22.74583596 │ 78.9    │   66.0 │   5.36 │
│ 201101190013 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 18.6723886  │ 57.3    │  105.0 │   3.82 │
│ 201101190014 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 28.08433211 │ 79.2    │   48.0 │   5.07 │
│ 201101190074 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      90.0 │ 25.20398137 │ 88.8    │  131.0 │   6.39 │
│ 201101190110 │ 19/1/2011 0:00  │  179.

In [ ]:
duckdb.sql('SELECT AVG(age) FROM data WHERE age > 60')

┌─────────────────┐
│    avg(age)     │
│     double      │
├─────────────────┤
│ 66.640081799591 │
└─────────────────┘

In [ ]:
duckdb.sql('SELECT AVG(age) FILTER (age > 60), AVG(age), MEDIAN(age), MIN(age), MAX(age) FROM data')

┌────────────────────────────────────┬───────────────────┬─────────────┬──────────┬──────────┐
│ avg(age) FILTER (WHERE (age > 60)) │     avg(age)      │ median(age) │ min(age) │ max(age) │
│               double               │      double       │   double    │  double  │  double  │
├────────────────────────────────────┼───────────────────┼─────────────┼──────────┼──────────┤
│                    66.640081799591 │ 46.72509254788347 │        49.0 │      0.0 │     93.0 │
└────────────────────────────────────┴───────────────────┴─────────────┴──────────┴──────────┘

In [ ]:
duckdb.sql('''SELECT AVG(age) FILTER (age > 60) AS avg_aging_age,
AVG(age) AS avg_age,
MEDIAN(age) AS median_age,
MIN(age) AS min_age,
MAX(age) AS max_age
FROM data''').df()

,avg_aging_age,avg_age,median_age,min_age,max_age
0,66.640082,46.725093,49.0,0.0,93.0


## Python Relational Operation API

[More examples](https://github.com/duckdb/duckdb/blob/main/examples/python/duckdb-python.py)

[DuckDBPyRelation Reference](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyRelation)

In [ ]:
rel = duckdb.df(data) # create a relational object

In [ ]:
rel.columns

['ServiceNo',
 'ServiceDate',
 'CustID',
 'age',
 'sex',
 'cmsCode',
 'Weight',
 'Height',
 'HeartRate',
 'Systolic',
 'Diastolic',
 'BMI',
 'HDL',
 'TG',
 'UA']

In [ ]:
rel.describe()

┌─────────┬────────────────────┬────────────────┬───┬─────────┬────────────────────┬───────────────────┐
│  aggr   │     ServiceNo      │  ServiceDate   │ … │   HDL   │         TG         │        UA         │
│ varchar │       double       │    varchar     │   │ varchar │       double       │      double       │
├─────────┼────────────────────┼────────────────┼───┼─────────┼────────────────────┼───────────────────┤
│ count   │            18645.0 │ 18645          │ … │ 15196   │            15254.0 │           13476.0 │
│ mean    │ 201106133252.34137 │ NULL           │ … │ NULL    │ 121.12829421791005 │  5.37851142772333 │
│ stddev  │  2801075.903464601 │ NULL           │ … │ NULL    │  92.70951522975376 │ 1.454882671327019 │
│ min     │     201101050001.0 │ 1/11/2011 0:00 │ … │ 100     │               21.0 │              0.66 │
│ max     │     201112280131.0 │ 9/8/2011 0:00  │ … │ ไม่มีเลือด │             1877.0 │             13.14 │
│ median  │     201106230063.0 │ NULL           │ … 

In [ ]:
rel.show()

┌──────────────┬────────────────┬────────┬────────┬────────┬───┬───────────┬─────────────┬─────────┬────────┬────────┐
│  ServiceNo   │  ServiceDate   │ CustID │  age   │  sex   │ … │ Diastolic │     BMI     │   HDL   │   TG   │   UA   │
│    int64     │    varchar     │ double │ double │ double │   │  double   │   varchar   │ varchar │ double │ double │
├──────────────┼────────────────┼────────┼────────┼────────┼───┼───────────┼─────────────┼─────────┼────────┼────────┤
│ 201101050001 │ 5/1/2011 0:00  │  225.0 │   59.0 │    2.0 │ … │      96.0 │ 32.35631502 │ 56.4    │  115.0 │   5.04 │
│ 201101050002 │ 5/1/2011 0:00  │  225.0 │   55.0 │    2.0 │ … │      70.0 │ 22.06607495 │ 73.7    │  141.0 │   3.82 │
│ 201101050003 │ 5/1/2011 0:00  │  225.0 │   60.0 │    2.0 │ … │      70.0 │ 23.5042735  │ 77.5    │   69.0 │   5.01 │
│ 201101050004 │ 5/1/2011 0:00  │  225.0 │   55.0 │    2.0 │ … │      80.0 │ 24.34175829 │ 77.9    │   79.0 │   2.83 │
│ 201101050005 │ 5/1/2011 0:00  │  225.0 │   59.

In [ ]:
rel.count('*') # counts all rows

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        18645 │
└──────────────┘

In [ ]:
rel.count('ServiceNo, age, weight, height')

┌──────────────────┬────────────┬───────────────┬───────────────┐
│ count(ServiceNo) │ count(age) │ count(weight) │ count(height) │
│      int64       │   int64    │     int64     │     int64     │
├──────────────────┼────────────┼───────────────┼───────────────┤
│            18645 │      18639 │         17439 │         17444 │
└──────────────────┴────────────┴───────────────┴───────────────┘

In [ ]:
rel.filter('age > 60.0')

┌──────────────┬─────────────────┬────────┬────────┬────────┬───┬───────────┬─────────────┬─────────┬────────┬────────┐
│  ServiceNo   │   ServiceDate   │ CustID │  age   │  sex   │ … │ Diastolic │     BMI     │   HDL   │   TG   │   UA   │
│    int64     │     varchar     │ double │ double │ double │   │  double   │   varchar   │ varchar │ double │ double │
├──────────────┼─────────────────┼────────┼────────┼────────┼───┼───────────┼─────────────┼─────────┼────────┼────────┤
│ 201101050010 │ 5/1/2011 0:00   │  225.0 │   61.0 │    2.0 │ … │      80.0 │ 22.74583596 │ 78.9    │   66.0 │   5.36 │
│ 201101190013 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 18.6723886  │ 57.3    │  105.0 │   3.82 │
│ 201101190014 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 28.08433211 │ 79.2    │   48.0 │   5.07 │
│ 201101190074 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      90.0 │ 25.20398137 │ 88.8    │  131.0 │   6.39 │
│ 201101190110 │ 19/1/2011 0:00  │  179.

In [ ]:
rel.filter('age > 60.0').count('age,weight,height')

┌────────────┬───────────────┬───────────────┐
│ count(age) │ count(weight) │ count(height) │
│   int64    │     int64     │     int64     │
├────────────┼───────────────┼───────────────┤
│       1956 │          1594 │          1594 │
└────────────┴───────────────┴───────────────┘

In [ ]:
rel.filter('age > 60.0').limit(10)

┌──────────────┬────────────────┬────────┬────────┬────────┬───┬───────────┬─────────────┬─────────┬────────┬────────┐
│  ServiceNo   │  ServiceDate   │ CustID │  age   │  sex   │ … │ Diastolic │     BMI     │   HDL   │   TG   │   UA   │
│    int64     │    varchar     │ double │ double │ double │   │  double   │   varchar   │ varchar │ double │ double │
├──────────────┼────────────────┼────────┼────────┼────────┼───┼───────────┼─────────────┼─────────┼────────┼────────┤
│ 201101050010 │ 5/1/2011 0:00  │  225.0 │   61.0 │    2.0 │ … │      80.0 │ 22.74583596 │ 78.9    │   66.0 │   5.36 │
│ 201101190013 │ 19/1/2011 0:00 │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 18.6723886  │ 57.3    │  105.0 │   3.82 │
│ 201101190014 │ 19/1/2011 0:00 │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 28.08433211 │ 79.2    │   48.0 │   5.07 │
│ 201101190074 │ 19/1/2011 0:00 │  179.0 │   61.0 │    2.0 │ … │      90.0 │ 25.20398137 │ 88.8    │  131.0 │   6.39 │
│ 201101190110 │ 19/1/2011 0:00 │  179.0 │   61.

In [ ]:
rel.filter('age > 60.0').project('age,BMI,HDL,TG').limit(10)

┌────────┬─────────────┬─────────┬────────┐
│  age   │     BMI     │   HDL   │   TG   │
│ double │   varchar   │ varchar │ double │
├────────┼─────────────┼─────────┼────────┤
│   61.0 │ 22.74583596 │ 78.9    │   66.0 │
│   61.0 │ 18.6723886  │ 57.3    │  105.0 │
│   61.0 │ 28.08433211 │ 79.2    │   48.0 │
│   61.0 │ 25.20398137 │ 88.8    │  131.0 │
│   61.0 │ 29.20332937 │ 58.3    │  133.0 │
│   61.0 │ 19.52240591 │ 84.9    │   84.0 │
│   62.0 │ 22.9719382  │ 72.4    │   79.0 │
│   74.0 │ 25.14285714 │ 46.9    │  131.0 │
│   61.0 │ 26.89232314 │ 37.7    │  198.0 │
│   68.0 │ 26.00906435 │ 46.7    │  125.0 │
├────────┴─────────────┴─────────┴────────┤
│ 10 rows                       4 columns │
└─────────────────────────────────────────┘

In [ ]:
rel.filter('age > 60.0').avg('age')

┌─────────────────┐
│    avg(age)     │
│     double      │
├─────────────────┤
│ 66.640081799591 │
└─────────────────┘

In [ ]:
rel.filter('age > 60.0').std('age')

┌───────────────────┐
│ stddev_samp(age)  │
│      double       │
├───────────────────┤
│ 5.489660944203022 │
└───────────────────┘

In [ ]:
duckdb.filter(data, 'age > 60.0') # shorthand version

┌──────────────┬─────────────────┬────────┬────────┬────────┬───┬───────────┬─────────────┬─────────┬────────┬────────┐
│  ServiceNo   │   ServiceDate   │ CustID │  age   │  sex   │ … │ Diastolic │     BMI     │   HDL   │   TG   │   UA   │
│    int64     │     varchar     │ double │ double │ double │   │  double   │   varchar   │ varchar │ double │ double │
├──────────────┼─────────────────┼────────┼────────┼────────┼───┼───────────┼─────────────┼─────────┼────────┼────────┤
│ 201101050010 │ 5/1/2011 0:00   │  225.0 │   61.0 │    2.0 │ … │      80.0 │ 22.74583596 │ 78.9    │   66.0 │   5.36 │
│ 201101190013 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 18.6723886  │ 57.3    │  105.0 │   3.82 │
│ 201101190014 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      80.0 │ 28.08433211 │ 79.2    │   48.0 │   5.07 │
│ 201101190074 │ 19/1/2011 0:00  │  179.0 │   61.0 │    2.0 │ … │      90.0 │ 25.20398137 │ 88.8    │  131.0 │   6.39 │
│ 201101190110 │ 19/1/2011 0:00  │  179.

In [ ]:
rel.filter('age > 60').select('age, weight, height')

┌────────┬─────────┬─────────┐
│  age   │ Weight  │ Height  │
│ double │ varchar │ varchar │
├────────┼─────────┼─────────┤
│   61.0 │ 55      │ 155.5   │
│   61.0 │ 40.9    │ 148     │
│   61.0 │ 71      │ 159     │
│   61.0 │ 59      │ 153     │
│   61.0 │ 61.4    │ 145     │
│   61.0 │ 45.7    │ 153     │
│   62.0 │ 51      │ 149     │
│   74.0 │ 77      │ 175     │
│   61.0 │ 75      │ 167     │
│   68.0 │ 67      │ 160.5   │
│     ·  │ ·       │  ·      │
│     ·  │ ·       │  ·      │
│     ·  │ ·       │  ·      │
│   68.0 │ 86.3    │ 162     │
│   66.0 │ 67.4    │ 147.5   │
│   63.0 │ 64.5    │ 157     │
│   70.0 │ 55.5    │ 155     │
│   72.0 │ 58      │ 155     │
│   63.0 │ 62.5    │ 154     │
│   65.0 │ 61      │ 165.5   │
│   63.0 │ 83      │ 155.5   │
│   61.0 │ 53.5    │ 146     │
│   64.0 │ 75      │ 165     │
├────────┴─────────┴─────────┤
│    1956 rows (20 shown)    │
└────────────────────────────┘

In [ ]:
rel.filter('age > 60').select('age, weight, height').sql_query()

'SELECT age, weight, height FROM (SELECT * FROM (SELECT * FROM pandas_scan(0x7cf98b0f43d0)) AS df_20beee09b5c977d4 WHERE (age > 60)) AS df_20beee09b5c977d4'

In [ ]:
import sqlite3

In [ ]:
sqlite_conn = sqlite3.connect('hospital.db')

In [ ]:
doctors = pd.read_sql_query('SELECT * FROM doctors', con=sqlite_conn)

In [ ]:
departments = pd.read_sql_query('SELECT * FROM departments', con=sqlite_conn)

In [ ]:
r_doctors = duckdb.df(doctors).set_alias('d')
r_depts = duckdb.df(departments).set_alias('p')

In [ ]:
doctor_depts = r_doctors.join(r_depts, 'd.department_code=p.code')

In [ ]:
doctor_depts.show()

┌───────┬───────────┬────────────┬────────────┬─────────────────┬─────────┬───────────────────┐
│  id   │ firstname │  lastname  │  license   │ department_code │  code   │       name        │
│ int64 │  varchar  │  varchar   │  varchar   │     varchar     │ varchar │      varchar      │
├───────┼───────────┼────────────┼────────────┼─────────────────┼─────────┼───────────────────┤
│     1 │ Larry     │ Patel      │ 0953979718 │ MED             │ MED     │ Internal Medicine │
│     2 │ Joshua    │ Holloway   │ 4226068794 │ ER              │ ER      │ Emergency         │
│     3 │ Tammy     │ Richardson │ 7075041268 │ OTH             │ OTH     │ Orthopedics       │
│     4 │ Amanda    │ Andersen   │ 3151389665 │ MED             │ MED     │ Internal Medicine │
│     5 │ Mary      │ Drake      │ 8084236287 │ GYN             │ GYN     │ Gynecology        │
│     6 │ Katherine │ Harper     │ 0883872318 │ GYN             │ GYN     │ Gynecology        │
│     7 │ Amanda    │ Fernandez  │ 02789

In [ ]:
doctor_depts.value_counts('code', groups='code')

┌─────────┬─────────────┐
│  code   │ count(code) │
│ varchar │    int64    │
├─────────┼─────────────┤
│ MED     │           5 │
│ ER      │           8 │
│ GYN     │          12 │
│ OTH     │           5 │
└─────────┴─────────────┘

In [ ]:
doctor_depts.create('doctor_dept_table')

In [ ]:
duckdb.sql('''
SELECT count(*), code FROM doctor_dept_table
GROUP BY code
ORDER BY count(*) DESC
''')

┌──────────────┬─────────┐
│ count_star() │  code   │
│    int64     │ varchar │
├──────────────┼─────────┤
│           12 │ GYN     │
│            8 │ ER      │
│            5 │ MED     │
│            5 │ OTH     │
└──────────────┴─────────┘

**Exercise**: What are the top 5 doctors who ordered most?

In [ ]:
lab_orders = pd.read_sql_query('SELECT * FROM lab_orders;', con=sqlite_conn)

In [ ]:
r_lab_orders = duckdb.df(lab_orders).set_alias('o')

In [ ]:
doctor_orders = r_doctors.join(r_lab_orders, 'd.id=o.doctor_id')

In [ ]:
doctor_orders.create('doctor_order_table')

In [ ]:
duckdb.sql('''
SELECT count(*) AS num_orders,firstname,lastname FROM doctor_order_table
GROUP BY firstname,lastname
ORDER BY count(*) DESC
LIMIT 10
''')

┌────────────┬───────────┬───────────┐
│ num_orders │ firstname │ lastname  │
│   int64    │  varchar  │  varchar  │
├────────────┼───────────┼───────────┤
│          9 │ Lauren    │ Allen     │
│          9 │ Sonia     │ Simpson   │
│          8 │ Jason     │ Thompson  │
│          8 │ Seth      │ Walton    │
│          8 │ Larry     │ Patel     │
│          7 │ Katherine │ Harper    │
│          7 │ Amanda    │ Fernandez │
│          7 │ Ashley    │ Barr      │
│          6 │ Patricia  │ Martin    │
│          6 │ Jane      │ Miller    │
├────────────┴───────────┴───────────┤
│ 10 rows                  3 columns │
└────────────────────────────────────┘

In [ ]:
doctor_orders.value_counts('firstname,lastname', groups='firstname,lastname').sort('count(firstname)').sort('count(lastname)')

┌───────────┬────────────┬──────────────────┬─────────────────┐
│ firstname │  lastname  │ count(firstname) │ count(lastname) │
│  varchar  │  varchar   │      int64       │      int64      │
├───────────┼────────────┼──────────────────┼─────────────────┤
│ Catherine │ Spencer    │                1 │               1 │
│ Daniel    │ Guzman     │                1 │               1 │
│ Nancy     │ Travis     │                2 │               2 │
│ Walter    │ Knapp      │                2 │               2 │
│ Sheri     │ Bowen      │                2 │               2 │
│ Charles   │ Brooks     │                3 │               3 │
│ Anthony   │ Jones      │                3 │               3 │
│ Miranda   │ Williams   │                3 │               3 │
│ Tammy     │ Richardson │                3 │               3 │
│ Mary      │ Drake      │                3 │               3 │
│  ·        │   ·        │                · │               · │
│  ·        │   ·        │              